# NOTEBOOK 6: POWER BI DATAPREP

# 🧠 Model Results Preparation for Power BI Dashboard

This notebook serves as the final transformation layer to prepare the test dataset for visualization in Power BI. The primary goals are:

- Load the trained ML model and saved threshold  
- Predict churn probabilities on the **unseen test set**  
- Convert those probabilities into meaningful business labels (Yes/No)  
- Calculate **Revenue at Risk** per customer  
- Restore original (unscaled) numerical features for clean visualization  
- Rebuild human-readable labels from dummy variables (`Contract`, `InternetService`, `PaymentMethod`)  
- Convert all boolean columns to integer format to avoid Power BI issues  
- Tag each prediction as "Correct" or "Wrong" by comparing with actual churn  
- Assign **Churn Risk Buckets** for executive-friendly segmentation  
- Export a clean, final dataset for seamless integration into Power BI

> This notebook bridges our machine learning model with our business-facing Power BI dashboard, allowing us to communicate model predictions, insights, and risks clearly to non-technical stakeholders.


In [ ]:
import pandas as pd
import numpy as np
import joblib

model = joblib.load(r"C:\Users\ADMIN\Documents\GitHub\Customer-churn-prediction\models/rf_churn_model.pkl")
threshold = joblib.load(r"C:\Users\ADMIN\Documents\GitHub\Customer-churn-prediction\models/rf_threshold.pkl")  # should be 0.503
X_test = pd.read_csv(r"C:\Users\ADMIN\Documents\GitHub\Customer-churn-prediction\data\X_test.csv")
X_test_unscaled = pd.read_csv(r"C:\Users\ADMIN\Downloads\X_test_raw.csv")[['MonthlyCharges', 'tenure', 'TotalCharges']]
y_test = pd.read_csv(r"C:\Users\ADMIN\Documents\GitHub\Customer-churn-prediction\data\y_test.csv")

# Predict churn probabilities
X_test["PredictedChurnProb"] = model.predict_proba(X_test)[:, 1]

# Apply threshold to get binary churn prediction
X_test["PredictedChurn"] = (X_test["PredictedChurnProb"] >= threshold).astype(int)

# Optional: Make it more readable
X_test["PredictedChurnLabel"] = X_test["PredictedChurn"].map({1: "Yes", 0: "No"})

# Calculate Revenue at Risk
X_test["RevenueAtRisk"] = X_test["PredictedChurnProb"] * X_test_unscaled["MonthlyCharges"]
X_test["RevenueAtRisk"] = X_test["RevenueAtRisk"].round(2)

#Replace num columns with 
X_test['MonthlyCharges'] = X_test_unscaled['MonthlyCharges']
X_test['tenure'] = X_test_unscaled['tenure']
X_test['TotalCharges'] = X_test_unscaled['TotalCharges']

Dashboard_set = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
Dashboard_set.rename(columns={'Churn': 'ActualChurn'}, inplace=True)

Dashboard_set["CorrectPrediction"] = (Dashboard_set["PredictedChurn"] == Dashboard_set["ActualChurn"]).map({True: "Correct", False: "Wrong"})
# Find all boolean columns
bool_cols = Dashboard_set.select_dtypes(include='bool').columns

# Convert them to integers (0/1)
Dashboard_set[bool_cols] = Dashboard_set[bool_cols].astype(int)




print(Dashboard_set.head())
print(Dashboard_set.dtypes)


In [ ]:
# Rebuild Contract column
Dashboard_set['Contract'] = Dashboard_set[['Contract_One year', 'Contract_Two year']].idxmax(axis=1)
Dashboard_set['Contract'] = Dashboard_set['Contract'].fillna('Contract_Month-to-month')
Dashboard_set['Contract'] = Dashboard_set['Contract'].map({
    'Contract_One year': 'One Year',
    'Contract_Two year': 'Two Year',
    'Contract_Month-to-month': 'Month-to-Month'
})


Dashboard_set['InternetService'] = Dashboard_set[['InternetService_Fiber optic', 'InternetService_No']].idxmax(axis=1)
Dashboard_set['InternetService'] = Dashboard_set['InternetService'].fillna('InternetService_DSL')
Dashboard_set['InternetService'] = Dashboard_set['InternetService'].map({
    'InternetService_Fiber optic': 'Fiber optic',
    'InternetService_No': 'No',
    'InternetService_DSL': 'DSL'
})
Dashboard_set['PaymentMethod'] = Dashboard_set[[
    'PaymentMethod_Credit card (automatic)',
    'PaymentMethod_Electronic check',
    'PaymentMethod_Mailed check'
]].idxmax(axis=1)

Dashboard_set['PaymentMethod'] = Dashboard_set['PaymentMethod'].fillna('PaymentMethod_Bank transfer (automatic)')

Dashboard_set['PaymentMethod'] = Dashboard_set['PaymentMethod'].map({
    'PaymentMethod_Credit card (automatic)': 'Credit Card (Auto)',
    'PaymentMethod_Electronic check': 'Electronic Check',
    'PaymentMethod_Mailed check': 'Mailed Check',
    'PaymentMethod_Bank transfer (automatic)': 'Bank Transfer (Auto)'
})

Dashboard_set.drop(columns=[
    'Contract_One year', 'Contract_Two year',
    'InternetService_Fiber optic', 'InternetService_No',
    'PaymentMethod_Credit card (automatic)',
    'PaymentMethod_Electronic check',
    'PaymentMethod_Mailed check'
], inplace=True)



In [ ]:
def assign_risk(prob):
    if prob < 0.34:
        return 'Low Risk'
    elif prob < 0.67:
        return 'Medium Risk'
    else:
        return 'High Risk'

Dashboard_set['ChurnRiskBucket'] = Dashboard_set['PredictedChurnProb'].apply(assign_risk)

Dashboard_set['ChurnRiskOrder'] = Dashboard_set['ChurnRiskBucket'].map({
    'Low Risk': 0,
    'Medium Risk': 1,
    'High Risk': 2
})



In [ ]:
Dashboard_set.to_csv(r"C:\Users\ADMIN\Documents\GitHub\Customer-churn-prediction\data/dashboard_dataset.csv", index=False)


#  Final Outputs

### we achieved the following:

1. Predicted churn probabilities on the **test set** using the trained model  
2. Applied custom decision threshold (0.503) to convert probabilities into binary predictions  
3. Calculated **Revenue at Risk** by multiplying churn probability with Monthly Charges  
4. Restored original values of `MonthlyCharges`, `tenure`, and `TotalCharges` for clean visuals  
5. Reconstructed readable labels for:
- `Contract` (One Year, Two Year, Month-to-Month)
- `InternetService` (Fiber Optic, DSL, No)
- `PaymentMethod` (Credit Card, Mailed Check, etc.)

🔹 Converted all boolean columns to 0/1 integers (to prevent Power BI filter issues)  
🔹 Added a column to show **Correct vs Wrong Predictions**  
🔹 Created **Churn Risk Buckets**: Low, Medium, High  
🔹 Exported a **Power BI-ready dataset**: clean, labeled, business-aligned

> We're now ready to build a visually powerful and analytically sound dashboard to showcase your model's value and business insights.
